In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time
import warnings

warnings.filterwarnings( 'ignore' )
os.environ[ 'TF_CPP_MIN_LOG_LEVEL' ] = '3'
os.environ[ 'TF_FORCE_GPU_ALLOW_GROWTH ' ] = '1'

import tensorflow as tf
import tensorflow.keras.models as km
import tensorflow.keras.layers as kl
import tensorflow.keras.losses as kls
import tensorflow.keras.optimizers as koz

import tensorflow_addons as tfa
import tensorflow_probability as tfp

import numpy as np
import gym

from pathlib import Path

import model as m


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth( gpu, True )
    logical_gpus = tf.config.experimental.list_logical_devices( 'GPU' )
    print( len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs", logical_gpus )
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


In [4]:
# set up gym and reset

    
env = gym.make( 
    'gym_fxcm:fxcm-v5',
    bias_profit=False,
    bias_action=0, 
    bias_runner=False, 
    dataset_file='../../../lambda-asset-corpus/ALL/XAUUSD_1M_ALL.csv', 
)

print( 'dataset size, ', len(env.dataset) )

ZZ  ZT
2009-03-15 17:00:00  929.60   0
2009-03-18 11:36:00  883.65  -1
2009-03-20 04:02:00  967.55   1
2009-04-06 18:00:00  861.00  -1
2009-04-26 20:34:00  918.88   1
dataset size,  3447353


In [5]:
# create model and model agent

    
model = m.Model( 
    num_actions=env.action_space.n, 
    num_observations=env.observation_space.shape[0] 
)
agent = m.A2CAgent( model )


In [6]:
# train model - force correct action during training
batch_sz = 2400 # average length of target trade over ten years
updates = int( len( env.dataset ) / batch_sz )
    
_ = agent.train( 
    env, 
    batch_sz=batch_sz, 
    override_action_value=False,
    updates=updates, 
    update_frequency=10,
    update_graph_path='./graph/'
)


 **    SHORT: 1445 | profit: $ -00.00 | lots: -1 | open: 923.5 | close: 923.5 | length: 0 | drawdown: $ 000.00 | balance: $ 10,000.00
CLOSE       : 1446 | profit: $ 000.10 | lots: -1 | open: 923.5 | close: 923.4 | length: 1 | drawdown: $ 000.00 | balance: $ 9,999.75
 **     LONG: 1447 | profit: $ 000.00 | lots: 1 | open: 923.4 | close: 923.4 | length: 0 | drawdown: $ 000.00 | balance: $ 9,999.75
CLOSER      : 1448 | profit: $ 000.00 | lots: 0 | open: 923.4 | close: 923.45 | length: 1 | drawdown: $ 000.00 | balance: $ 9,999.45
 **    SHORT: 1456 | profit: $ -00.00 | lots: -1 | open: 923.6 | close: 923.6 | length: 0 | drawdown: $ 000.00 | balance: $ 9,999.45
CLOSER      : 1459 | profit: $ 000.00 | lots: 0 | open: 923.6 | close: 923.7 | length: 3 | drawdown: $ -00.10 | balance: $ 9,999.00
 **    SHORT: 1460 | profit: $ -00.00 | lots: -1 | open: 924.25 | close: 924.25 | length: 0 | drawdown: $ -00.10 | balance: $ 9,999.00
 **    SHORT: 1462 | profit: $ -00.03 | lots: -2 | open: 924.265 | c

In [ ]:
agent.model.summary()
